Initial Setup

In [1]:
import sys
sys.path.append('Libraries/')
import pclpy
import cv2
import numpy as np
import pdal
import segTree
import Utils
import pandas as pd
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import Tree_Tool

from tkinter import *
import tkinter.ttk as ttk
import tkinter as tk
import os

#Utils
def tic():
    global timestart
    timestart = time.perf_counter()
    
def toc():
    global timestart
    return time.perf_counter()-timestart

from tkinter import *
import tkinter.ttk as ttk
import tkinter as tk
from tkinter import filedialog
import threading
import open3d as o3d

import open3d as o3d
import time

In [2]:
def onClossing():
    root.quit()
    root.destroy()

In [3]:
def process(PointCloudV):
    global Mysegmentor
    
    Mysegmentor = Tree_Tool.Tree_tool(PointCloudV)
    Mysegmentor.set_Ksearch(0.08)
    Mysegmentor.Full_Process()

In [4]:
def directory():
    global directory, Mysegmentor
    directory = filedialog.askopenfile()
    PointCloud = pclpy.pcl.PointCloud.PointXYZ()
    pclpy.pcl.io.loadPCDFile(directory.name,PointCloud)
    PointCloudV = segTree.voxelize(PointCloud.xyz,0.04)
    
    lockView = threading.Lock()
    process(PointCloudV)
    Utils.open3dpaint(PointCloudV, reduce_for_Vis = True , pointsize=0.16)

In [5]:
def Main():
    root.after(10, Main)

def step1():
    Utils.open3dpaint([Mysegmentor.NongroundCloud,Mysegmentor.GroundCloud],reduce_for_Vis = True , voxelsize=0.2)

def step2():
    Utils.open3dpaint([Mysegmentor.NongroundCloud.xyz, Mysegmentor.nonFilteredpoints.xyz + Mysegmentor.nonFilterednormals * 0.1,
                          Mysegmentor.nonFilteredpoints.xyz + Mysegmentor.nonFilterednormals * 0.2],reduce_for_Vis = True , voxelsize=0.2)

def step3():
    Utils.open3dpaint([Mysegmentor.filteredpoints.xyz , Mysegmentor.filteredpoints.xyz + Mysegmentor.filterednormals * 0.05,
                          Mysegmentor.filteredpoints.xyz + Mysegmentor.filterednormals * 0.1],reduce_for_Vis = True , voxelsize=0.2)

def step4():
    Utils.open3dpaint(Mysegmentor.cluster_list,reduce_for_Vis = True , voxelsize=0.2)

def step5():
    Utils.open3dpaint(Mysegmentor.complete_Stems,reduce_for_Vis = True , voxelsize=0.2)

def step6():
    Utils.open3dpaint(Mysegmentor.lowstems,reduce_for_Vis = True , voxelsize=0.2)

def step7():
    Utils.open3dpaint([i['tree'] for i in Mysegmentor.finalstems] + Mysegmentor.stemcyls,reduce_for_Vis = True , voxelsize=0.2)
    
def step8():
    Utils.open3dpaint(Mysegmentor.lowstems + [Mysegmentor.Pointcloud.xyz*0.98],reduce_for_Vis = True ,pointsize=2., voxelsize=0.2)
    
def step9():
    Utils.open3dpaint(Mysegmentor.lowstems + [Mysegmentor.filteredpoints.xyz*0.98],reduce_for_Vis = True ,pointsize=2., voxelsize=0.2)

In [ ]:
Mysegmentor = []

root = Tk()
root.protocol('WM_DESTROY_WINDOW', onClossing)
root.title('Controles')

label0 = Button(root, text = 'Step 1',command= step1)
label0.grid(row=0, column=1, padx=5, pady=5)
label1 = Button(root, text = 'Step 2',command= step2)
label1.grid(row=0, column=2, padx=5, pady=5)
label2 = Button(root, text = 'Step 3',command= step3)
label2.grid(row=0, column=3, padx=5, pady=5)
label3 = Button(root, text = 'Step 4',command= step4)
label3.grid(row=0, column=4, padx=5, pady=5)
label4 = Button(root, text = 'Step 5',command= step5)
label4.grid(row=1, column=1, padx=5, pady=5)
label5 = Button(root, text = 'Step 6',command= step6)
label5.grid(row=1, column=2, padx=5, pady=5)
label6 = Button(root, text = 'Step 7',command= step7)
label6.grid(row=1, column=3, padx=5, pady=5)
label6 = Button(root, text = 'Step 8',command= step8)
label6.grid(row=1, column=4, padx=5, pady=5)
label6 = Button(root, text = 'Step 9',command= step9)
label6.grid(row=2, column=1, padx=5, pady=5)


buttonDir = Button(root, text='Selección del Directorio', command = directory)
buttonDir.grid(row = 0, column = 0, padx = 20, pady = 20)

Main()

root.mainloop()

Step_1_Remove_Floor
Step_2_Normal_Filtering
Step_3_Eucladean_Clustering
Step_4_Group_Stems
Step_5_Get_Ground_Level_Trees
Step_6_Get_Cylinder_Tree_Models
Done


In [ ]:
#####################################################
#Get ground truth
treedata = pd.read_csv('D:/Documentos/ParaAutonomia/Python/Proyecto/Data/EuroSDR_DataRelease/EuroSDR_DataRelease/TLS_Benchmarking_Plot_3_LHD.txt',sep = '\t',names = ['x','y','height','DBH'])
Xcor,Ycor,diam = treedata.iloc[0,[0,1,3]]
surtreesL1 = [Utils.makecylinder(model=[Xcor, Ycor, 0,0,0,1,diam/2],length=10,dense=20)]
Zcor = 0
TreeDict = [np.array([Xcor,Ycor,diam])]
for i,rows in treedata.iloc[1:].iterrows():
    Xcor,Ycor,diam = rows.iloc[[0,1,3]]
    if not np.any(np.isnan([Xcor,Ycor,diam])):
        surtreesL1.append(Utils.makecylinder(model=[Xcor, Ycor, 0,0,0,1,diam/2],length=10,dense=10))
        TreeDict.append(np.array([Xcor,Ycor,diam]))
surtrees1 = [p for i in surtreesL1 for p in i]
surtreesCloud1 = pclpy.pcl.PointCloud.PointXYZ(surtrees1)

#DataBase
#Found trees
#Hungarian Algorithm assignment
CostMat = np.ones([len(TreeDict),len(Mysegmentor.stemcyls)])
for X,datatree in enumerate(TreeDict):
    for Y,foundtree in enumerate(Mysegmentor.finalstems):
        CostMat[X,Y] = np.linalg.norm([datatree[0:2]-foundtree['model'][0:2]])

dataindex, foundindex = linear_sum_assignment(CostMat,maximize=False)

#Get metrics
locationerror = []
correctlocationerror = []
diametererror = []
cloudmatch = []
for i,j in zip(dataindex, foundindex):
    locationerror.append(np.linalg.norm((Mysegmentor.finalstems[j]['model'][0:2]-TreeDict[i][0:2])))
    if locationerror[-1]<0.6:
        diametererror.append(abs(Mysegmentor.finalstems[j]['model'][6]*2-TreeDict[i][2]))        
        correctlocationerror.append(np.linalg.norm((Mysegmentor.finalstems[j]['model'][0:2]-TreeDict[i][0:2])))
    cloudmatch.append(np.vstack([surtreesL1[i],Mysegmentor.finalstems[j]['tree'],Mysegmentor.stemcyls[j]]))

#PCA filtering and Eucladian Clustering
Visualization.PCL3dpaint(cloudmatch)